In [0]:
!pip install -I pillow==5.1.0

In [0]:
# this sets up the virtual environment if gdrive doesn't contain the repo clone
# it by setting the the if to true and modifie the path as you wish
#@title Default title text
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
import os
!pip install -q xlrd
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/ADL4CV')
#!ls
if False:
  from getpass import getpass
  user = getpass('GitHub user')
  password = getpass('GitHub password')
  os.environ['GitHub_AUTH'] = user + ':' + password
  !git clone https://$GitHub_AUTH@github.com/Cram13/ADL4CV_Project.git
os.chdir('ADL4CV_Project')
!git pull origin master
#!pip install -I pillow==5.1.0
!pip3 install torch==0.4.1 torchvision==0.2.1
!pip3 install tensorboardX
#!pip install PIL
#!pip install image
#import PIL.image

In [0]:
 #!git checkout origin/master
!git pull origin master
#!git status
#!git fetch
#!git reset --hard origin/master

In [0]:
#setup options for training
from train_yolo_example import Opt
from torch.utils.data import DataLoader, Subset
from yolo.yolo_LSTM import YoloLSTM
from yolo.loss import YoloLoss
from train_yolo_lstm import load_Snapshot_to_yolo_LSTM#, filter_gt #,train
import torch

from dataset_utils.datasets.MotBBImageSequence import *
from torch.utils.data import DataLoader, Subset
from yolo.yolo_LSTM import YoloLSTM
from yolo.loss import YoloLoss
from train_yolo_example import Opt
import torch.nn as nn
import torch
from yolo.yolo_encoder import YoloEncoder
from yolo.yolo_lstm_part import YoloLSTM_part
from tensorboardX import SummaryWriter
import shutil
from train_yolo_example import writeLossToSummary
from yolo.yolo_utils import *


In [0]:
#fire up tensorboard in the background
#todo make path first
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format('./log/')#opt.log_path)
)
get_ipython().system_raw('/content/ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:

def filter_non_zero_gt(gt):
    # used to filter only existing bb for loss
    return gt[gt[:, 0] != 0.0][:, 1:]

'''opt = Opt()
opt.batch_size = 4
opt.num_epoches = 100
opt.useCuda = True
opt.learning_rate = 1e-6
opt.num_workers = 4
anchor = [(6.88, 27.44), (11.93, 58.32), (19.90, 94.92), (40.00, 195.84), (97.96, 358.62)]
anchor[:] = [(x[0] / 32.0, x[1]/32.0) for x in anchor]
opt.model = Yolo(0, anchors=anchor)
loadYoloBaseWeights(opt.model, opt)
opt.criterion = yloss(0, opt.model.anchors, opt.reduction)'''
opt = Opt()
opt.useCuda = True
opt.learning_rate = 1e-4
opt.batch_size = 4
anchors = [(0.43, 1.715), (0.745625, 3.645), (1.24375, 5.9325), (2.5, 12.24), (6.1225, 22.41375)]
opt.model = YoloLSTM(opt.batch_size, image_size=832, anchors=anchors)
opt.pre_trained_yolo_path = './log/2_yolo_832/snapshot0025.tar'
opt.image_size = 832
#load_Snapshot_to_yolo_LSTM(opt.model.encoder, opt)
opt.model.load_pretrained_weights(opt.pre_trained_yolo_path)
device = torch.device("cuda")
opt.model.to(device)
opt.criterion = YoloLoss(opt.model.anchors, filter_fkt=filter_non_zero_gt)
opt.num_workers = 0

opt.log_path = './log/test_marc99_continue'
#train(opt)

In [0]:
path = './log/test_marc99/snapshot0004.tar'
device = next(opt.model.parameters()).device
opt.model.load_state_dict(torch.load(path, map_location=device)['model_state_dict'])
opt.optimizer = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999), weight_decay=opt.decay)
opt.optimizer.load_state_dict(torch.load(path, map_location=device)['optimizer_state_dict'])
#for param_group in opt.optimizer.param_groups:
#  param_group['lr'] = 1e-4
#opt.model.image_size = 832
#opt.image_size = 832

In [0]:
 
def train(opt):
  # setup train and eval set
  if torch.cuda.is_available() and opt.useCuda:
      torch.cuda.manual_seed(123)
  else:
      torch.manual_seed(123)
  dataset = MotBBImageSequence('dataset_utils/Mot17_test_single.txt', use_only_first_video=False, new_height=832, new_width=832)
  train_data = Subset(dataset, range(0, dataset.valid_begin))
  valid_data = Subset(dataset, range(dataset.valid_begin, len(dataset)))
  train_loader = DataLoader(train_data, batch_size=opt.batch_size, shuffle=True, num_workers=opt.num_workers)#, drop_last=True)
  valid_loader = DataLoader(valid_data, batch_size=opt.batch_size, shuffle=False, num_workers=opt.num_workers)#, drop_last=True)

  # log stuff
  if os.path.isdir(opt.log_path):
      shutil.rmtree(opt.log_path)
  os.makedirs(opt.log_path)
  writer = SummaryWriter(opt.log_path)
  #fix this so that we get our graph
  #if torch.cuda.is_available() and opt.useCuda:
  #    writer.add_graph(opt.model.cpu(), torch.rand(opt.batch_size, 3, opt.image_size, opt.image_size))
  #    opt.model.cuda()
  #else:
  #    writer.add_graph(opt.model, torch.rand(opt.batch_size, 3, opt.image_size, opt.image_size))
  writer.add_text('Hyperparams', 'lr: {}, \nbatchsize: {}, \nimg_size:{}'.format(opt.learning_rate, opt.batch_size, opt.image_size))

  opt.optimizer = torch.optim.Adam(opt.model.parameters(), lr=opt.learning_rate, betas=(opt.momentum, 0.999),weight_decay=opt.decay)
  epoch_len = len(train_loader)
  print('epoch_len: {}'.format(epoch_len))
  for epoch in range(opt.num_epoches):
    print('num epoch: {:4d}'.format(epoch))
    opt.model.train()
    for img_nr, (gt, b, img, d) in enumerate(train_loader):
      sequence_len = gt.shape[1]
      opt.model.reinit_lstm(gt.shape[0])
      opt.optimizer.zero_grad()
      seq_loss, seq_loss_coord, seq_loss_conf = (0.0, 0.0, 0.0)
      for i in range(sequence_len):
        if torch.cuda.is_available() and opt.useCuda:
          single_gt = gt[:, i].cuda()
          single_img = img[:, i].cuda()
        else:
          single_gt = gt[:, i]
          single_img = img[:, i]
        #print(single_gt.shape)
        #print(filter_non_zero_gt(single_gt[0]).shape)
        logits = opt.model(single_img*255.)
        loss, loss_coord, loss_conf = opt.criterion(logits, single_gt)
        seq_loss += loss
        seq_loss_conf += loss_conf.item()
        seq_loss_coord += loss_coord.item()
      seq_loss.backward()#retain_graph=True)
      opt.optimizer.step()
      writeLossToSummary(writer, 'Train', seq_loss.item()/sequence_len, seq_loss_coord/sequence_len,
                         seq_loss_conf/sequence_len, epoch * epoch_len + img_nr)


    # eval stuff
    opt.model.eval()
    valid_len = len(valid_loader)
    loss_eval = 0
    loss_coord_eval = 0
    loss_conf_eval = 0
    all_ap = []
    all_ap1 = []
    for img_nr, (gt, b, img, d) in enumerate(valid_loader):
      sequence_len = gt.shape[1]
      opt.model.reinit_lstm(gt.shape[0])
      seq_loss, seq_loss_coord, seq_loss_conf = (0.0, 0.0, 0.0)
      for i in range(sequence_len):
        if torch.cuda.is_available() and opt.useCuda:
          single_gt = gt[:, i].cuda()
          single_img = img[:, i].cuda()
        else:
          single_gt = gt[:, i]
          single_img = img[:, i]
        with torch.no_grad():
          logits = opt.model(single_img*255.)
          loss, loss_coord, loss_conf = opt.criterion(logits, single_gt)
          for i in range(single_gt.shape[0]):
              ap = get_ap(logits[i], filter_non_zero_gt(single_gt[i]), opt.image_size, opt.image_size,
                          opt.model.anchors, .5)
              ap1 = get_ap(logits[i], filter_non_zero_gt(single_gt[i]), opt.image_size, opt.image_size,
                           opt.model.anchors, .8)
              if not np.isnan(ap):
                  all_ap.append(ap)
              if not np.isnan(ap1):
                  all_ap1.append(ap1)
        seq_loss += loss.item()
        seq_loss_conf += loss_conf.item()
        seq_loss_coord += loss_coord.item()

      loss_eval += seq_loss/sequence_len
      loss_coord_eval += seq_loss_coord/sequence_len
      loss_conf_eval += seq_loss_conf/sequence_len


    writeLossToSummary(writer, 'Val', loss_eval/valid_len, loss_coord_eval/valid_len,
                       loss_conf_eval/valid_len, (epoch+1) * epoch_len)

    writer.add_scalar('Val/AP0.5', np.mean(np.array(all_ap)), epoch * epoch_len)
    writer.add_scalar('Val/AP0.8', np.mean(np.array(all_ap1)), epoch * epoch_len)
    print(np.mean(np.array(all_ap)))
    #print('{}  {}   {}'.format(loss_eval/valid_len, loss_coord_eval/valid_len,
    #                           loss_conf_eval/valid_len))

    torch.save({'epoch': epoch, 'model_state_dict': opt.model.state_dict(),
                'optimizer_state_dict': opt.optimizer.state_dict()},
               opt.log_path+'/snapshot{:04d}.tar'.format(epoch))

  writer.close()

In [0]:
train(opt)

In [0]:
import time
for i in range(3600):
  time.sleep(1)